# Introduction to Solving QUBO Problems on a D-Wave Quantum Annealer

Quantum annealing is an alternative type of quantum computing, also referred to as adiabatic, or "analog", quantum computing. Whereas "digital" quantum computing allows for the development of quantum-gate based algorithms (circuits) for general-purpose problems, quantum annealing is specifically suited for **quadratic unconstrained binary optimization (QUBO)** problems.

### 1. What is a QUBO?

Despite the similarity, the acronym QUBO, by itself, has nothing to do with "qubits" or "quantum"! A QUBO (Quadratic Unconstrained Binary Optimization) is a classical optimization problem, consisting in **minimizing** a quadratic cost function of the form $$ f(\{x_i\})\ =\ \sum_{i\leq j = 1}^n \,Q_{i,j}\,x_i\,x_j\ =\ \mathbf{x}^{\top}\mathbf{Q}\,\mathbf{x}$$ where $\{x_i\}$ is a set of binary variables, i.e., $x_i \in \{0,1\}$. This type of problem is often encountered in combinatorics for finance/economics, one famous example is the *travelling salesman* problem, see Exercise 1. The same mathematical form is also frequently found in optimization problems for materials science, as we will see in Exercise 2. It is interesting to note that binary variables fulfill $x_i = x_i^2$. Therefore, the QUBO cost function can also be written in the form $$ f(\{x_i\})\ =\ \sum_{i = 1}^n \,Q_{i,i}\,x_i + \sum_{i<j = 1}^n \,Q_{i,j}\,x_i\,x_j$$ with linear and quadratic terms.

Classical computer algorithms to solve QUBO problems typically scale exponentially with the dimension $n$ of the variable space. This makes QUBOs with large $n$ very difficult to solve on classical computers. Quantum annealing offers an attractive alternative to solving classical QUBO problems.

### 2. What is Quantum Annealing?

#### 2.a.  Mapping a classical QUBO onto a quantum spin (qubit) network

Like in other types of quantum computing, quantum annealing is a method to solving computational problems by manipulating, and finally measuring, the quantum state of a multi-qubit system. A qubit is represented by a quantum-mechanical two-level system with eigenstates $|{1}\rangle$ and $|{0}\rangle$. A quantum annealer consists of a network of physically coupled qubits with tunable coupling strengths. The D-Wave Advantage System has over 5000 qubits!

 <img src="qubit_network.pdf" width="300"> *Representation of a quantum annealer as a network of qubits with tunable coupling strengths (arrows).*

The coupled qubit network is described by the Ising model for a network of spin variables $\sigma_i\in\{|{\uparrow}\rangle,|{\downarrow}\rangle\}$, which is represented by a Hamiltonian $$ \mathcal{H}_z = \sum_i h_i\,\hat{\sigma}_z^{(i)} + \sum_{i<j} J_{i,j}\,\hat{\sigma}_z^{(i)}\hat{\sigma}_z^{(j)} $$ The QUBO cost function described in section 1. can be directly mapped onto the spin Hamiltonian by identifying $h_i \rightarrow Q_{i,i}$ and $J_{i,j} \rightarrow Q_{i,j}$. Here, we interpret the state $\sigma_i = |{\uparrow}\rangle$ as $x_i = 1$ and $\sigma_i = |{\downarrow}\rangle$ as $x_i = 0$. In other words: **The ground state of the spin (qubit) network with such Hamiltonian is directly equivalent to the solution of the corresponding (classical) QUBO minimization problem.** We can therefore solve the QUBO problem by determining the ground state of the qubit network with a suitable Hamiltonian.

#### 2.b.  Solving a classical QUBO by quantum annealing

The coupling strengths of the physical qubit network in a quantum annealer can be tuned, e.g., by applying certain magnetic fields. This allows to implement the desired Hamiltonian. However, finding the corresponding ground state is still challenging. For this, the **adiabatic theorem** of quantum mechanics is employed: If a system is subject to a Hamiltonian $\mathcal{H}(t)$ with a "very" slow time dependence and the system is in the initial ground state $|{\Psi_0(t=0)}\rangle$ of $\mathcal{H}(t=0)$, then the system will adiabatically remain in the (time-dependent) ground state of $\mathcal{H}(t)$.

This behavior can be exploited for solving the QUBO problem: The many-qubit system is first prepared in the easily achievable ground state $|{\Psi_0}\rangle$ of some "easy" initial Hamiltonian $\mathcal{H}_0$. Then the coupling strengths are "slowly" adjusted to adiabatically turn $\mathcal{H}_0$ into the desired $\mathcal{H}_z$ encoding our QUBO problem, $\mathcal{H}_0 \longrightarrow \mathcal{H}_z$. If the process is sufficicently slow, then the system remains in the corresponding ground state, yielding the ground state of $\mathcal{H}_z$ as final state, $|{\Psi_0}\rangle \longrightarrow |{\Psi_{\mathrm{sol}}}\rangle$. By measuring the final state of the system, we thus directly obtain the solution of the classical QUBO problem, e.g., $|{\Psi_{\mathrm{sol}}}\rangle = |{0,1,1,1,0,1,0,0,0,1,1,...}\rangle$. The process of slowly (adiabatically) turning the "easy" initial (ground) state into the final ground state of the target Hamiltonian is called **quantum annealing**.

### 3. Quantum Annealing in Practice: The D-Wave Leap Platform

The Canadian company D-Wave Quantum Systems Inc. is a pioneer in the development of practically usable quantum annealers. The company operates the D-Wave Leap cloud platform providing access to quantum annealers for solving real QUBO problems. You have already seen a real D-Wave Advantage system during the guided tour at Forschungszentrum Jülich. So let's use this quantum computer...

**Go to the D-Wave Leap Website and set up your personal D-Wave Leap account** with a free trial minute of QPU time (QPU: Quantum Processing Unit)! After logging in, you will find yourself on the Dashboard. Here you will **find your personal token**, which you will need to authenticate for accessing quantum annealing resources. So, copy it to the clipboard!

<img src="token_DWave_Leap.png" width="200">

#### 3.a. Create a new workspace for the tutorial

Quantum annealing resources (also called "solvers") are conveniently included in Python scripts using methods from the D-Wave module for Python. To do this, we start a new workspace...**click on "IDE Workspaces" at the top of the Dashboard, then select "New Workspace", type the URL of the tutorial's GitHub repository into the field "CONTEXT URL", and create the workspace**. You will now find the Jupyter Notebooks of the tutorial in the directory of the workspace...ready to get started! <br> **Start the notebooks by typing, e.g., "jupyter notebook 000_introduction.ipynb" into the terminal.**

#### 3.b. The basic structure of a Python script for D-Wave

We will now construct the most basic version of a Python script for solving a QUBO problem on a D-Wave quantum annealer. First, we need to import some D-Wave packages...

In [ ]:
from dwave.system import DWaveSampler, EmbeddingComposite
import dwave.inspector

...and some useful standard packages...

In [ ]:
import numpy as np

This function will tell the system your authentication token.

In [ ]:
def get_token():
    return 'COPY YOUR PERSONAL TOKEN HERE'

We next need to define our QUBO problem. There are several options how to implement and pass the coefficients $Q_{i,j}$ of the QUBO matrix to the D-Wave solver method. One possibility is in the form of a Python dictionary. For this, we define the following routine:

In [ ]:
def get_qubo(QMat):
    
    matShape = np.array(QMat).shape
    
    Q = {}
    for i in range(matShape[0]):
        for j in range(matShape[1]):
            Q[(i, j)] = QMat[i][j]

    return Q

Let's test it...

In [ ]:
QMat = [[1,-3],
        [0,1]]

Q = get_qubo(QMat)

print(Q)

We are now ready for our first run on a real quantum computer! We need to first define the sampling method we wish to use. The D-Wave package provides different options, including, or even combining, classical sampling methods and quantum annealing. "EmbeddingComposite(DWaveSampler())" returns the quantum annealer as sampler.

In [ ]:
# We choose the quantum annealer as solver for our QUBO problem
sampler = EmbeddingComposite(DWaveSampler())

# And we make 10 independent annealing runs...
numruns = 10
sample_set = sampler.sample_qubo(Q, num_reads=numruns)

print(sample_set)

The D-Wave Inspector provides a graphical interface to examine the solutions returned from the annealing runs.

In [ ]:
dwave.inspector.show(sample_set)

Some remarks on the terminology: 
- The terms "sampler" and "solver" are used interchangeably. With "solver", we express that we wish to solve our QUBO problem, i.e., find the state with the lowest value of the cost function. The term "sampler" acknowledges that in reality, we will not always obtain the global minimum solution from each annealing run. The annealing output has a cetain statistical distribution, depending on the complexity of the QUBO problem. We will come back to this point later.
- The value of the cost function is often referred to simply as "energy", even in cases where the QUBO problem by itself has nothing to do with physics or energy. The reason is that any QUBO problem will be encoded in a certain Hamiltonian, which effectively describes the energy of the multi-qubit system. As described above, the quantum annealing process results in the ground state, i.e., lowest-energy state, of the target Hamiltonian.

#### 3.c. **Task:** Change the entries of the QUBO matrix, run the problem on the quantum annealer, and investigate the results

You can test, e.g., the following matrices, but feel free to play around...

In [ ]:
[[1,2],
[0,1]]

In [ ]:
[[1,-2],
[0,1]]

In [ ]:
[[-1,-2],
[0,-1]]

Can you explain the sampling output?